# Evaluation Results of Rare Adverse Drug Reactions (ADRs) in OFFSIDES

Import necessary modules: 

In [1]:
import numpy as np
import pandas as pd
from ADRprofilePrediction import Pairs2Mat, evaluation
from Models import loadHyperpar
import json

In [2]:
import sklearn
print(sklearn.__version__)

0.24.2


## Load data

Load the feature data in to a dictionary. Drug-target, drug-enzyme, drug-chemical structure fingerprint, drug-gene interaction, drug-transporter, drug-pathway and drug-indication are included.

In [3]:
features_dict = {
    "target":Pairs2Mat(path="data/drug_target.tsv",colname1="0",colname2="1"),
    "enzyme":Pairs2Mat(path="data/drug_enzyme.tsv",colname1="0",colname2="1"),
    "Chem":pd.read_csv("data/drug_chemsfp.tsv",sep = "\t",header=0,index_col=0),
    "DGI":Pairs2Mat(path="data/interactions.tsv",colname1="drug_claim_name",colname2="gene_name"),
    "transporter":Pairs2Mat(path="data/drug_transporter.tsv",colname1="0",colname2="1"),
    "pathway":Pairs2Mat(path="data/drug_pathway.tsv",colname1="0",colname2="1"),
    "indication":Pairs2Mat(path="data/drug_indication.tsv",colname1="1_x",colname2="6")
}


Load ADR data from SIDER and OFFSIDES. Variable SEs is a dict that stores ADR data. Variable filter controls the frequency of the ADR. When filter is "all", only the ADRs with extremely low frequencies are removed; when filter is "rare" only frequency less than 50 were used.


In [4]:
filter = "rare"
SEs = {}
if filter == "all":
    SIDER = Pairs2Mat(path="data/drug_se.tsv",colname1="1_x",colname2="5")
    column_sums = np.sum(SIDER, axis=0)
    SEs["SIDER"] = SIDER.loc[:, (column_sums >= 5)]

    OFFSIDERS = Pairs2Mat(path="data/OFFSIDES.csv",colname1="drug_concept_name",colname2="condition_concept_name",sep = ",")
    column_sums = np.sum(OFFSIDERS, axis=0)
    SEs["OFFSIDES"] = OFFSIDERS.loc[:, column_sums >= 5]
elif filter == "rare":
    SIDER = Pairs2Mat(path="data/drug_se.tsv",colname1="1_x",colname2="5")
    column_sums = np.sum(SIDER, axis=0)
    SEs["SIDER"] = SIDER.loc[:, (column_sums < 50)]

    OFFSIDERS = Pairs2Mat(path="data/OFFSIDES.csv",colname1="drug_concept_name",colname2="condition_concept_name",sep = ",")
    column_sums = np.sum(OFFSIDERS, axis=0)
    SEs["OFFSIDES"] = OFFSIDERS.loc[:, column_sums < 50]


## Set variables

The variables below includes all the options for the code.

- features_names: This varible is the list of all the features including the target feature, the enzyme feature, chemical structure fingerprint (Chem), drug-gene interaction (DGI), the transporter featrue, the pathway feature, the indication feature.
- SE_names: ADR data from SIDER.
- methods: This option is machine learning methods used for prediction, including Smoothed Kernel Regression (SKR), Kernel Regression (KR), the na\"ive method (Naive), Linear Neighbourhood Similarity Method using Regularized Linear Neighbour Similarity or Jaccard similarity (LNSM_RLN, LNSM_jaccard), Support Vector Machine (SVM), Random Forest (RF) and Boosted Random Forest (BRF).
- metrice_names: Metric we used to evaluate the performance of methods: AUPR, AUROC, AUPR per drug, AUROC per drug, AUPR+AUROC and AUPR+AUROC per drug.
- SE_name: The used ADR data in this file.
- metric: We used AUPR as the tuning metrice in Nested CV and CV.

In [7]:
features_names = ["target", "enzyme", "Chem", "DGI", "transporter", "pathway", "indication"]
# SEs_names = ["SIDER", "OFFSIDES"]
# methods = ["SKR", "KR", "KRR", "Naive", "LNSM_RLN", "LNSM_jaccard", "VKR"]
methods = ["SKR", "KRR", "VKR", "Naive", "LNSM_RLN", "LNSM_jaccard"]
# methods = ["SKR", "KR", "KRR", "Naive", "LNSM_RLN", "LNSM_jaccard", "VKR", "SVM", "OCCA", "SCCA", "RF", "BRF"]
tuning_metrices=["AUROC", "AUPR", "AUROCperdrug", "AUPRperdrug"]
metrice_names = ["AUPR+AUROC", "AUPR+AUROCperdrug", "AUROC", "AUPR", "AUROCperdrug", "AUPRperdrug"]
SEs_name = "OFFSIDES"
metrice = "AUPR"

Set the variables for hyperparameters. We summarized 3 types of hyperparameters (SVM, RF and BRF are not competitive and time-consuming, and were tuned and trained in a seperated file -- SVM_RF.ipynb): 
 - A: This hyperparameters are tuned according to the step $\dots, 10^{-1}, 10^{0}, 10^{1}, \dots$ ($\sigma_X$ and $\sigma_Y$ does not change during tuning so they can be set as $10$ and $100$ respectively).
 - B: This hyperparameters are in $[0,1]$ and tuned according to the step $0, 0.1, \dots, 1$.
 - C: This hyperparameters are tuned based on $5, 10, 15, \dots$.

In [8]:
A = 10**np.arange(-2, 3, 1, dtype=float)
B = np.arange(0.1, 1, 0.1, dtype=float)
C = np.arange(5, 20, 5, dtype=int)
A10 = 10**np.arange(1, 2, 1, dtype=float)
A100 = 10**np.arange(2, 3, 1, dtype=float)
all_hyperparlist = {
    "SKR":[A,B,A10,A100], 
    # "KR":[A,A], 
    "KRR":[A,A],
    "VKR":[A,A,C], 
    "Naive":[], 
    "LNSM_RLN":[B,A], 
    "LNSM_jaccard":[B], 
    # "SVM":[A,A,A], 
    # "OCCA":[], 
    # "SCCA":[A], 
    # "RF":[C], 
    # "BRF":[C]
}

Set dictionaries to store the tuned hyperparameters and the results of CV and Nested CV.

In [10]:
hyperparsOut = {}
hyperparsOut["nested_cv"] = {}
hyperparsOut["cv"] = {}
results = {}
results["nested_cv"] = {}
results["cv"] = {}

## Nested CV and CV

Load tuned hyperparameters. If fully rerunning the tuning step of Nested CV and CV is required, please skip loading variable hyperpars and remove the option `hyperparfixed` of the function `evaluation()`.

In [17]:
# Open and read the JSON file
with open(f'results/hyperpars_{SEs_name}.xml', 'r') as xml_file:
    hyperpars = json.load(xml_file)

In [18]:
for method in methods:
    # # method = "SKR"
    validation = "nested_cv"
    hyperparsOut[validation][method] = {}
    results[validation][method] = {}
    for str in features_names:
        print(f"using feature {str}")
        hyperparList = loadHyperpar(*all_hyperparlist[method],method_option=method)
        results[validation][method][str], hyperparsOut[validation][method][str] = evaluation(Y=SEs[SEs_name], X=features_dict[str], method_option=method,tuning_metrice=metrice,hyperparList=hyperparList,hyperparfixed=hyperpars[validation][method][str],Validation=validation,n_jobs=1)

    # method = "SKR"
    validation = "cv"
    hyperparsOut[validation][method] = {}
    results[validation][method] = {}
    for str in features_names:
        print(f"using feature {str}")
        hyperparList = loadHyperpar(*all_hyperparlist[method],method_option=method)
        results[validation][method][str], hyperparsOut[validation][method][str] = evaluation(Y=SEs[SEs_name], X=features_dict[str], method_option=method,tuning_metrice=metrice,hyperparList=hyperparList,hyperparfixed=hyperpars[validation][method][str],Validation=validation,n_jobs=1)

using feature target
The SKR requires hyperparameter lambda, c, sigma_X, sigma_Y
---------- nested cv start ----------
Fold: 0
number of hyperpars combination:  45
first few training idx:  [ 35 107 184 194 251 328 410 461 578 653]
first few testing idx:  [224 236 273 301 306 322 406 434 435 585]
--- tuning end ---
target size: 133
------ best hyper pars:  [0.01, 0.6, 10, 100] ------
SKR starts:
SKR ends:
-----------
AUPRperdrug: 0.06660713799659151
AUROCperdrug: 0.763499214292935
AUPR+AUROCperdrug: 0.8301063522895265
AUPR: 0.05529583154927727
AUROC: 0.7158406525058227
AUPR+AUROC: 0.7711364840551
-----------
Fold: 1
number of hyperpars combination:  45
first few training idx:  [224 236 273 301 306 322 406 434 435 585]
first few testing idx:  [ 35 107 184 194 251 328 410 461 578 653]
--- tuning end ---
target size: 133
------ best hyper pars:  [0.01, 0.6, 10, 100] ------
SKR starts:
SKR ends:
-----------
AUPRperdrug: 0.045205174465070705
AUROCperdrug: 0.7311265260518489
AUPR+AUROCperdrug

## Save Output

Store the tuned hyperparameters for reproducing results.

In [ ]:
# with open(f'results/hyperpars_{SEs_name}.xml', 'w') as xml_file:
#    json.dump(hyperparsOut, xml_file)

Store the results of Nested CV and CV.

In [19]:
with open(f'results/results_{SEs_name}_{filter}.xml', 'w') as xml_file:
   json.dump(results, xml_file)

## Reorganize the Results and Calculate the P-value of Method Comparison

Load the results of Nested CV and CV.

In [20]:
with open(f'results/results_{SEs_name}_{filter}.xml', 'r') as xml_file:
    results = json.load(xml_file)

Orgainize the results of Nested CV into table.

In [21]:
df = pd.DataFrame()
for m, fs in results["nested_cv"].items():
    for f, mes in fs.items():
        for me, scores in mes.items():
            temp_df = pd.DataFrame({
                'method': m,
                'feature': f,
                'metric': me,
                "score": scores
            })
            df = pd.concat([df, temp_df], ignore_index=True)

custom_order = ["pathway","Chem", "DGI",  "indication", "target", "transporter", "enzyme"]
df['feature'] = pd.Categorical(df['feature'], categories=custom_order, ordered=True)
df['method'] = pd.Categorical(df['method'], categories=methods, ordered=True)
df['metric'] = pd.Categorical(df['metric'], categories=metrice_names, ordered=True)
df2 = pd.pivot_table(df, values=['score'], index=["feature", "method"], aggfunc={'score': ["mean","std"]}, columns=["metric"])
df3 = df2.sort_index(axis=1, level='metric').sort_index(level='feature')
df3.to_excel(f'results/nested_cv_results_{SEs_name}_{filter}.xlsx')
df3

score                               \
                               mean        std              mean   
metric                   AUPR+AUROC AUPR+AUROC AUPR+AUROCperdrug   
feature     method                                                 
pathway     SKR            0.814269   0.018117          0.871404   
            KRR            0.805827   0.042154          0.862769   
            VKR            0.820921   0.027405          0.805101   
            Naive          0.767696   0.010109          0.804008   
            LNSM_RLN       0.673099   0.019942          0.675122   
            LNSM_jaccard   0.568672   0.059902          0.613726   
Chem        SKR            0.754919   0.004893          0.799146   
            KRR            0.760468   0.011330          0.799096   
            VKR            0.748777   0.023003          0.763515   
            Naive          0.768181   0.008198          0.791657   
            LNSM_RLN       0.737925   0.028304          0.776185   
            LNSM_jaccard   0.494104   0.064808          0.509939   
DGI         SKR            0.795032   0.032403          0.766325   
            KRR            0.788402   0.033184          0.754145   
            VKR            0.822825   0.056709          0.737703   
            Naive          0.763230   0.004684          0.791271   
            LNSM_RLN       0.658938   0.087176          0.583780   
            LNSM_jaccard   0.584726   0.072568          0.586484   
indication  SKR            0.828867   0.026429          0.798170   
            KRR            0.822033   0.027201          0.786478   
            VKR            0.858318   0.033817          0.755545   
            Naive          0.766308   0.008823          0.786497   
            LNSM_RLN       0.810816   0.038656          0.726233   
            LNSM_jaccard   0.830529   0.043140          0.849507   
target      SKR            0.762840   0.010292          0.808843   
            KRR            0.774719   0.013200          0.806249   
            VKR            0.762659   0.022568          0.768708   
            Naive          0.764287   0.007091          0.788572   
            LNSM_RLN       0.534693   0.053209          0.544967   
            LNSM_jaccard   0.521428   0.049517          0.463201   
transporter SKR            0.758890   0.015971          0.781509   
            KRR            0.742939   0.036255          0.758639   
            VKR            0.750352   0.031282          0.746489   
            Naive          0.779139   0.008246          0.800313   
            LNSM_RLN       0.697937   0.037582          0.675653   
            LNSM_jaccard   0.502103   0.018276          0.496839   
enzyme      SKR            0.761056   0.019546          0.766486   
            KRR            0.756048   0.021462          0.756548   
            VKR            0.765077   0.035166          0.743081   
            Naive          0.764574   0.003980          0.788434   
            LNSM_RLN       0.694913   0.053412          0.638591   
            LNSM_jaccard   0.522075   0.028632          0.506503   

                                                                          \
                                       std      mean       std      mean   
metric                   AUPR+AUROCperdrug     AUROC     AUROC      AUPR   
feature     method                                                         
pathway     SKR                   0.013377  0.741110  0.020481  0.073159   
            KRR                   0.029783  0.731458  0.041290  0.074368   
            VKR                   0.018085  0.755380  0.018511  0.065542   
            Naive                 0.008040  0.728631  0.003547  0.039066   
            LNSM_RLN              0.017086  0.606184  0.018617  0.066915   
            LNSM_jaccard          0.047646  0.538100  0.058566  0.030572   
Chem        SKR                   0.009303  0.714800  0.012575  0.040120   
            KRR                   0.007808  0.721034  0.016006  0

In [22]:
df = pd.DataFrame()
for m, fs in results["cv"].items():
    for f, mes in fs.items():
        for me, scores in mes.items():
            temp_df = pd.DataFrame({
                'method': m,
                'feature': f,
                'metric': me,
                "score": scores
            },index=["1"])
            df = pd.concat([df, temp_df], ignore_index=True)
df['feature'] = pd.Categorical(df['feature'], categories=custom_order, ordered=True)
df['method'] = pd.Categorical(df['method'], categories=methods, ordered=True)
df['metric'] = pd.Categorical(df['metric'], categories=metrice_names, ordered=True)
df2 = pd.pivot_table(df, values=['score'], index=["feature", "method"], columns="metric")
df2.to_excel(f'results/cv_results_{SEs_name}_{filter}.xlsx')
df2

score                                        \
metric                   AUPR+AUROC AUPR+AUROCperdrug     AUROC      AUPR   
feature     method                                                          
pathway     SKR            0.767661          0.813291  0.717659  0.050001   
            KRR            0.767345          0.811571  0.717364  0.049981   
            VKR            0.763138          0.778752  0.713824  0.049315   
            Naive          0.770548          0.810164  0.719736  0.050811   
            LNSM_RLN       0.702373          0.690506  0.620436  0.081937   
            LNSM_jaccard   0.634257          0.625775  0.584585  0.049671   
Chem        SKR            0.798219          0.822046  0.741910  0.056309   
            KRR            0.780916          0.805757  0.735871  0.045044   
            VKR            0.770150          0.770245  0.727403  0.042748   
            Naive          0.773802          0.797056  0.730914  0.042888   
            LNSM_RLN       0.792761          0.819111  0.731410  0.061351   
            LNSM_jaccard   0.529088          0.533717  0.504267  0.024821   
DGI         SKR            0.833476          0.835969  0.761271  0.072205   
            KRR            0.772959          0.795644  0.730050  0.042909   
            VKR            0.807456          0.762960  0.749578  0.057877   
            Naive          0.759248          0.791190  0.721135  0.038113   
            LNSM_RLN       0.668318          0.611951  0.609683  0.058635   
            LNSM_jaccard   0.730844          0.693912  0.685904  0.044940   
indication  SKR            0.849747          0.869123  0.765105  0.084642   
            KRR            0.775213          0.803894  0.730775  0.044438   
            VKR            0.759624          0.771028  0.718291  0.041332   
            Naive          0.768696          0.798428  0.725313  0.043383   
            LNSM_RLN       0.826812          0.748810  0.719830  0.106982   
            LNSM_jaccard   0.513616          0.463805  0.485580  0.028036   
target      SKR            0.761298          0.800403  0.723482  0.037815   
            KRR            0.761802          0.800664  0.723754  0.038048   
            VKR            0.752825          0.766903  0.716564  0.036261   
            Naive          0.761134          0.795300  0.723270  0.037864   
            LNSM_RLN       0.654384          0.568209  0.594094  0.060290   
            LNSM_jaccard   0.583458          0.515137  0.547420  0.036038   
transporter SKR            0.783129          0.806742  0.732801  0.050327   
            KRR            0.779461          0.800410  0.730588  0.048873   
            VKR            0.771069          0.780166  0.723499  0.047570   
            Naive          0.776134          0.799114  0.727841  0.048293   
            LNSM_RLN       0.747343          0.679196  0.698733  0.048609   
            LNSM_jaccard   0.560009          0.536826  0.529118  0.030891   
enzyme      SKR            0.786742          0.803997  0.738925  0.047817   
            KRR            0.793786          0.802120  0.743106  0.050680   
            VKR            0.787551          0.780531  0.738128  0.049423   
            Naive          0.774607          0.801313  0.731106  0.043501   
            LNSM_RLN       0.796093          0.670204  0.736236  0.059857   
            LNSM_jaccard   0.501356          0.520768  0.477453  0.023903   

                                                   
metric                   AUROCperdrug AUPRperdrug  
feature     method                                 
pathway     SKR              0.760587    0.052704  
            KRR              0.759049    0.052522  
            VKR              0.727959    0.050793  
            Naive            0.757912    0.052252  
            LNSM_RLN         0.602370    0.088136  
            LNSM_jaccard     0.555283    0.070492  
Chem        SKR              0.766944    0.055103  
            KRR              0.758818    0.0

Calculate the P-value for method comparison based on the result of nested CV.

In [23]:
df = pd.DataFrame()
for m, fs in results["nested_cv"].items():
    for f, mes in fs.items():
        for me, scores in mes.items():
            temp_df = pd.DataFrame({
                'method': m,
                'feature': f,
                'metric': me,
            }, index=["1"])
            temp_df2 = pd.concat([temp_df, pd.DataFrame(scores, columns=["1"]).T], axis=1)
            df = pd.concat([df, temp_df2], ignore_index=True)
for m in metrice_names:
    for f in features_names:
        df2 = df[(df["metric"] == m) & (df["feature"] == f)]
        df3 = df2.iloc[:, np.array([0, 3, 4, 5, 6, 7])]
        df4 = df3.set_index(df3.columns[0])
        df5 = df4.T.ptests(paired=True, stars=False)
        df5.to_excel(f'results/pvalue_{SEs_name}_{filter}_{f}_{m}.xlsx')